In [22]:
import src.utils as utils
import os
import json

from dotenv import load_dotenv

load_dotenv()

db_config = {
    "dbname": "fpkg",
    "user": os.environ.get("PGUSER"),
    "password": os.environ.get("PGPASSWORD"),
    "host": "localhost",
    "port": "5431"
}

# Initialize database connection and get connection objects
conn, cursor, embeddings = utils.init_database(db_config)

Successfully initialized database and OpenAI embeddings


In [23]:
drug_of_interest = utils.vsearch_drug(drug="XANAX", cursor=cursor, conn=conn, embeddings=embeddings)

drug_id = drug_of_interest["id"]

found 5 results

Top result:
    Name: XANAX 0,25MG CPR
    ID: 13377



In [24]:
result = utils.get_contraindications(drug_id, cursor, conn)

In [25]:
print(json.dumps(result, indent=4))

[
    {
        "name": "HYPERSENSIBILITE",
        "id": "80943"
    },
    {
        "name": "INSUFFISANCE RESPIRATOIRE",
        "id": "80989"
    },
    {
        "name": "APNEE DU SOMMEIL SYNDROME",
        "id": "81300"
    },
    {
        "name": "INSUFFISANCE HEPATIQUE",
        "id": "80954"
    },
    {
        "name": "MYASTHENIE",
        "id": "81047"
    },
    {
        "name": "ADULTE JEUNE (15-17 ANS)",
        "id": "80962"
    },
    {
        "name": "ENFANT",
        "id": "80948"
    },
    {
        "name": "ENFANT",
        "id": "80948"
    },
    {
        "name": "NOURRISSON",
        "id": "80949"
    },
    {
        "name": "NOUVEAU-NE",
        "id": "80950"
    },
    {
        "name": "INTOXICATION ALCOOLIQUE AIGUE",
        "id": "81301"
    },
    {
        "name": "INTOXICATION MEDICAMENTEUSE OU CHIMIQUE",
        "id": "81182"
    },
    {
        "name": "ALLAITEMENT",
        "id": "80947"
    },
    {
        "name": "GROSSESSE",
        "id": "

In [31]:
import re

query = f"""
SELECT * FROM cypher('fcsv', $$ 
    MATCH ()-[r:HasIndication]->()
    RETURN r
$$, %(cypher_params)s) AS (x agtype);
"""

try:
    cursor.execute(query,params={
        "cypher_params": json.dumps(
            {
                "drug_id": drug_id,
            }
        )  
    })
    results = cursor.fetchall()
except Exception as e:
    print(e)
    conn.rollback()

for res in results[:20]:# Strip the text and load as json
    res = re.sub(r'::\w+$', '', res[0])
    res = json.loads(res)
    # Strip the text and load as json
    res = re.sub(r'::\w+$', '', res[0])
    res = json.loads(res)
    print(res)